In [1]:
reset()

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


In [2]:
import pandas as pd
import numpy as np


In [3]:
inputcsv=r'https://spainblob.blob.core.windows.net/altadis/Vending_Proportions.csv'

Popor_estancos = pd.read_csv(inputcsv) 


import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=ITE_PRD_esmz08srdb009;'
                      'Database=ITE_PRD;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
sql="""\

SELECT  a11.[CUSTOMER_ID]
	,case when a112.[Answer_Two] ='' then null else a112.[Answer_Two] end Sales2C
	,ceiling(isnull(case  a112.[Answer_Two] 
			  when '0% - 5%'   then  0.03
			  when '6% - 10%'  then  0.08
			  when '11% - 15%' then  0.13
			  when '16% - 20%' then  0.18
			  when '21% - 25%' then  0.23
			  when '26% - 30%' then  0.28
			  when '31% - 35%' then  0.33
			  when '36% - 40%' then  0.38
			  when '41% - 45%' then  0.43
			  when '46% - 50%' then  0.48
			  when '51% - 55%' then  0.53
			  when '56% - 60%' then  0.58
			  when '61% - 65%' then  0.63
			  when '66% - 70%' then  0.68
			  when '71% - 75%' then  0.73
			  when '76% - 80%' then  0.78
			  when '81% - 85%' then  0.83
			  when '86% - 90%' then  0.88
			  when '91% - 95%' then  0.93
			  when '96% - 100%' then 0.98
			  else 0 end,0) *100 )Sales2C_num
/*	,case when a112.[Answer_One] ='' then null else a112.[Answer_One] end tikets
	,case  a112.[Answer_One] 
	  when '<150' then  75           
	  when '>600' then  700
	  when 'Entre 151 y 250' then  200
	  when 'Entre 251 y 350' then  300
	  when 'Entre 351 y 400' then  375
	  when 'Entre 401 y 600' then  500
	ELSE 0 end tikets_num */
      , a13.[MES_ID]  MES
      , ISNULL(SUM(BLOND_VOL),0) [BLOND_VOL]   
      , ISNULL(SUM(BLOND_PCK),0) [BLOND_PCK]   
      , ISNULL(SUM(BLACK_VOL)/nullif(SUM(BLOND_VOL),0),0) [BLACK_VOL]   
      , ISNULL(SUM(BLACK_PCK)/nullif(SUM(BLOND_PCK),0),0) [BLACK_PCK]   
      , ISNULL(SUM(RYO_VOL)/nullif(SUM(BLOND_VOL),0),0) [RYO_VOL]       
      , ISNULL(SUM(RYO_PCK)/nullif(SUM(BLOND_PCK),0),0) [RYO_PCK]       
      , ISNULL(SUM(CIGAR_VOL)/nullif(SUM(BLOND_VOL),0),0) [CIGAR_VOL]   
      , ISNULL(SUM(PIPE_VOL)/nullif(SUM(BLOND_VOL),0),0) [PIPE_VOL]     
      , ISNULL(SUM(PIPE_PCK)/nullif(SUM(BLOND_PCK),0),0) [PIPE_PCK]     

  FROM [ITE_PRD].[ITE].[T_SMLM_CUSTTOTALS] a11  
  join ITE_PRD.ITE.LU_CLTE_1CANAL c on c.customer_id = a11.[Customer_ID]
	join	ITE.LU_MONTH	a12
	  on 	(a11.[SALESMONTH] = a12.[CAL_MONTH])
	join	ITE.V_Transf_MES_MQT1	a13
	  on 	(a12.[MES_ID] = a13.[MQT1])
left join	ITE.V_LU_CLTE_1CANAL_Assess	a112
	  on 	(a11.[CUSTOMER_ID] = a112.[CUSTOMER_ID] )	 
		and a112.[Nombre_Ficha] in (N'Datos Cuenta')
		and a112.[Assess_Status] in (N'Terminado')
        
        
 where  a13.[MES_ID] between 201600 and 201908
 and not (c.NAME like'%C.P%' or name like'C.I.%' or name like'CI %' or  NAME like'%centro penit%')
  and [CITY] not in ('CEUTA', 'MELILLA') --and STATUS_CODE <> '92'
 -- and case when a112.[Answer_One] ='' then null else a112.[Answer_One] end is not null
group by  a11.[CUSTOMER_ID]
	,a13.[MES_ID]
	,a112.[Answer_Two]
--	,a112.[Answer_One]

"""

Popor_estancos = pd.read_sql(sql,conn)

In [4]:
Popor_estancos.columns

Index(['CUSTOMER_ID', 'Sales2C', 'Sales2C_num', 'MES', 'BLOND_VOL',
       'BLOND_PCK', 'BLACK_VOL', 'BLACK_PCK', 'RYO_VOL', 'RYO_PCK',
       'CIGAR_VOL', 'PIPE_VOL', 'PIPE_PCK'],
      dtype='object')

In [5]:
Popor_estancos.head()

,CUSTOMER_ID,Sales2C,Sales2C_num,MES,BLOND_VOL,BLOND_PCK,BLACK_VOL,BLACK_PCK,RYO_VOL,RYO_PCK,CIGAR_VOL,PIPE_VOL,PIPE_PCK
0,28071074,31% - 35%,33.0,201603.0,1465576.0,71918.0,0.051310,0.052281,0.174444,0.082190,0.038343,0.003241,0.001320
1,31000094,0% - 5%,3.0,201706.0,558952.0,27914.0,0.145271,0.145088,0.184541,0.111986,0.080820,0.003935,0.001791
2,36150985,0% - 5%,3.0,201606.0,377128.0,18808.0,0.133111,0.133134,0.264276,0.158071,0.114536,0.009943,0.003881
3,47000104,16% - 20%,18.0,201805.0,331108.0,16493.0,0.123585,0.119808,0.114624,0.066634,0.079233,0.009030,0.004183
4,8000283,0% - 5%,3.0,201805.0,588280.0,29360.0,0.042578,0.041961,0.178362,0.103065,0.022795,0.004249,0.002043


In [6]:
feats_to_explore=['BLOND_VOL', 'BLOND_PCK', 'BLACK_VOL', 'BLACK_PCK', 'RYO_VOL', 'RYO_PCK', 'CIGAR_VOL', 'PIPE_VOL', 'PIPE_PCK']
#feats_to_explore=[ 'BLOND_PCK',  'BLACK_PCK', 'RYO_PCK', 'CIGAR_VOL', 'PIPE_PCK']
labels= ['0% - 5%', '6% - 10%', '11% - 15%', '16% - 20%', '21% - 25%', '26% - 30%', '31% - 35%', '36% - 40%', '41% - 45%', '46% - 50%', '51% - 55%', '56% - 60%','61% - 65%', '66% - 70%', '71% - 75%', '76% - 80%', '81% - 85%', '86% - 90%', '91% - 95%', '96% - 100%']


In [7]:
Popor_estancos.shape

(555618, 13)

In [8]:
from sklearn import preprocessing

lab_enc = preprocessing.LabelEncoder()
Score = lab_enc.fit_transform( Popor_estancos['Sales2C_num'])
Popor_estancos['Score']=Score


In [9]:
data=Popor_estancos

In [10]:
data=data.dropna()

In [11]:
data.shape

(443377, 14)

# Entrenamiento con muestra restrictiva.
Dado que el % de ventas al segundo canal es declarativo, tomare solamente los valores mas "fialbles", esto es eliminaré outliers de forma muy restritctiva, con estos datos, entrenare el modelo.

### dataset sin outliers

In [12]:
#dataset sin outliers

cat_Feat=['CUSTOMER_ID','Sales2C','Sales2C_num','Score','MES']
cat_data = data[cat_Feat]

num_data =data[feats_to_explore]


In [13]:
from scipy import stats

idx = np.all(stats.zscore(num_data) < 3, axis=1)
Data_cleaned = pd.concat([num_data.loc[idx], cat_data.loc[idx]], axis=1)

In [14]:
# Restrictive Tiles
Q1 = num_data.quantile(0.33)
Q3 = num_data.quantile(0.66)
IQR = Q3 - Q1
idx = ~((num_data < (Q1 - 1.5 * IQR)) | (num_data > (Q3 + 1.5 * IQR))).any(axis=1)
Data_cleaned = pd.concat([cat_data.loc[idx], num_data.loc[idx]], axis=1)

In [15]:
print (Data_cleaned.shape)
print('DataSet sin outliers:',round(100*Data_cleaned.shape[0]/ data.shape[0],2),'%')

(235091, 14)
DataSet sin outliers: 53.02 %


In [16]:
from sklearn.model_selection import train_test_split

Score = lab_enc.fit_transform( Data_cleaned['Sales2C_num'])

X_train, X_test, y_train, y_test = train_test_split(Data_cleaned[feats_to_explore], Score, test_size=0.1, random_state=2019, stratify=Data_cleaned['Sales2C_num'])

# RANDOM FOREST

In [20]:
print(n_estimators, ': Train Score: {:.2f}%'.format( rfc.score( X_train, y_train)*100))
print(n_estimators, ': out-of-bag Score: {:.2f}%'.format(rfc.oob_score_*100))
print(n_estimators, ': Test Score: {:.2f}%'.format(rfc.score(X_test, y_test)*100))

400 : Train Score: 99.86%
400 : out-of-bag Score: 52.88%
400 : Test Score: 53.80%


In [19]:
%%time
from sklearn.ensemble import RandomForestClassifier

#Create Classifier
n_estimators=400
rfc=RandomForestClassifier(n_estimators=n_estimators,  oob_score = True)
rfc.fit( X_train, y_train);

Wall time: 11min 31s


In [18]:
print(n_estimators, ': Train Score: {:.2f}%'.format( rfc.score( X_train, y_train)*100))
print(n_estimators, ': out-of-bag Score: {:.2f}%'.format(rfc.oob_score_*100))
print(n_estimators, ': Test Score: {:.2f}%'.format(rfc.score(X_test, y_test)*100))

300 : Train Score: 99.86%
300 : out-of-bag Score: 52.46%
300 : Test Score: 53.49%


In [22]:
X_train.columns

Index(['BLOND_VOL', 'BLOND_PCK', 'BLACK_VOL', 'BLACK_PCK', 'RYO_VOL',
       'RYO_PCK', 'CIGAR_VOL', 'PIPE_VOL', 'PIPE_PCK'],
      dtype='object')

# Predicción sobre el conjunto de datos completo

In [23]:
%%time
predic = rfc.predict(Popor_estancos[feats_to_explore])

Wall time: 4min 36s


In [24]:
predic.shape, Popor_estancos.shape

((555618,), (555618, 14))

In [25]:
Popor_estancos['predic_ID']=predic
Popor_estancos['predic']=lab_enc.inverse_transform(predic)

In [26]:
Popor_estancos[Popor_estancos.predic == Popor_estancos.Sales2C_num].count()/Popor_estancos.shape[0]

CUSTOMER_ID    0.506103
Sales2C        0.506103
Sales2C_num    0.506103
MES            0.506103
BLOND_VOL      0.506103
BLOND_PCK      0.506103
BLACK_VOL      0.506103
BLACK_PCK      0.506103
RYO_VOL        0.506103
RYO_PCK        0.506103
CIGAR_VOL      0.506103
PIPE_VOL       0.506103
PIPE_PCK       0.506103
Score          0.506103
predic_ID      0.506103
predic         0.506103
dtype: float64

In [27]:
Popor_estancos[cat_Feat+['predic']].head(30)

,CUSTOMER_ID,Sales2C,Sales2C_num,Score,MES,predic
0,28071074,31% - 35%,33.0,7,201603.0,33.0
1,31000094,0% - 5%,3.0,1,201706.0,8.0
2,36150985,0% - 5%,3.0,1,201606.0,8.0
3,47000104,16% - 20%,18.0,4,201805.0,3.0
4,8000283,0% - 5%,3.0,1,201805.0,3.0
5,33140534,NaN,0.0,0,201802.0,13.0
6,25010067,16% - 20%,18.0,4,201808.0,18.0
7,23040181,36% - 40%,38.0,8,201709.0,38.0
8,28060416,0% - 5%,3.0,1,201903.0,3.0
9,30070213,NaN,0.0,0,201602.0,8.0


In [34]:
Popor_estancos [Popor_estancos.CUSTOMER_ID==36002350][feats_to_explore+['predic']]

,BLOND_VOL,BLOND_PCK,BLACK_VOL,BLACK_PCK,RYO_VOL,RYO_PCK,CIGAR_VOL,PIPE_VOL,PIPE_PCK,predic
4558,531624.0,26512.0,0.073360,0.073551,0.215185,0.101388,0.089913,0.012405,0.003771,33.0
15033,539544.0,26852.0,0.089334,0.089751,0.269562,0.150789,0.092541,0.011398,0.004357,8.0
33244,449596.0,22359.0,0.076112,0.076523,0.222939,0.105281,0.076668,0.010654,0.003980,33.0
41994,550208.0,27420.0,0.069428,0.069657,0.242766,0.116119,0.090556,0.011450,0.003282,13.0
51937,411976.0,20524.0,0.072819,0.073085,0.252327,0.121370,0.090017,0.012379,0.003897,8.0
59310,481792.0,23972.0,0.074305,0.074670,0.238962,0.113382,0.083023,0.008489,0.003128,13.0
60821,531444.0,26487.0,0.074137,0.074376,0.235106,0.114055,0.087610,0.005626,0.001963,33.0
62597,527840.0,26296.0,0.068202,0.068451,0.214585,0.102829,0.096999,0.014701,0.004487,33.0
66395,520297.0,25891.0,0.068153,0.068479,0.222128,0.105905,0.082068,0.015875,0.004789,33.0
120521,513032.0,25546.0,0.074459,0.074767,0.219838,0.102716,0.091573,0.008771,0.002857,33.0


In [35]:
Output=r'C:\Users\esmijlmog\Documents\11111Proyectos11111\Industrializacion\Data\Vending_Predic_Prop.csv'

In [36]:
Popor_estancos[['CUSTOMER_ID', 'MES', 'Sales2C_num', 'predic']].to_csv(Output , sep='|', index=False) 


In [ ]:
#Duracion 7 min
    